<a href="https://colab.research.google.com/github/atul-ai/prompt-engineering-class/blob/main/TreePromptWithPromptChaining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prompt Chaining & Tree Prompting

We will first explore Prompt Chaining and then see an example of Tree Prompting.

We will as usual start with basic building block to install LangChain, Groq and initialize the llm.

## Initialization

In [1]:
!pip install langchain groq
!pip install langchain-groq groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.9/393.9 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.1/149.1 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [2]:
#Basic code to initialize the Groq API and setup the LLM.
from langchain_groq import ChatGroq
from langchain.chains import LLMChain
import os

# Set your Groq API key
os.environ["GROQ_API_KEY"] = ""

# Initialize the Groq LLM
llm = ChatGroq(model_name="mixtral-8x7b-32768")

## Prompt Chaining

We create a simple chain of three prompts, we write an introduction, provide three points about the introduction, expand on each of the points.

In [4]:
from langchain import PromptTemplate, LLMChain
from langchain.chains import SimpleSequentialChain

# First prompt template
first_prompt = PromptTemplate(
    input_variables=["topic"],
    template="Write a short introduction about {topic}."
)

# Second prompt template
second_prompt = PromptTemplate(
    input_variables=["introduction"],
    template="Based on this introduction, suggest 3 key points to expand upon: {introduction}"
)

# Third prompt template
third_prompt = PromptTemplate(
    input_variables=["key_points"],
    template="For each of these key points, provide a brief explanation: {key_points}"
)

# Create chains
chain_one = LLMChain(llm=llm, prompt=first_prompt)
chain_two = LLMChain(llm=llm, prompt=second_prompt)
chain_three = LLMChain(llm=llm, prompt=third_prompt)

# Create the sequential chain
overall_chain = SimpleSequentialChain(
    chains=[chain_one, chain_two, chain_three],
    verbose=True
)

# Run the chain
topic = "Artificial Intelligence"
result = overall_chain.run(topic)

print(result)



> Entering new SimpleSequentialChain chain...
Artificial Intelligence (AI) is a branch of computer science that focuses on creating intelligent machines that can think and learn like humans. The goal of AI is to enable machines to perform tasks that would normally require human intelligence, such as understanding natural language, recognizing patterns, solving problems, and making decisions. AI can be categorized into two main types: Narrow AI, which is designed to perform a specific task, and General AI, which has the ability to perform any intellectual task that a human being can do. AI has the potential to revolutionize various industries, including healthcare, finance, transportation, and education, by automating processes, improving efficiency, and providing new insights through data analysis. However, it also raises ethical and societal concerns, such as job displacement, privacy, and bias.
1. **Narrow AI vs. General AI:** It's important to understand the distinction between Na

## Tree Of Thought Prompting



This example demonstrates Tree of Thought Prompting using LangChain. It uses a few-shot learning approach to teach the model how to break down problems into steps. The key components are:

1. A base prompt template that encourages step-by-step thinking.
2. Examples of breaking down different types of questions.
3. A few-shot prompt template that combines the examples with the base prompt.

The model is then asked to approach a new question using this learned pattern of thought.

In [5]:
from langchain import PromptTemplate, LLMChain
from langchain.prompts import FewShotPromptTemplate

# Define the base prompt template
base_prompt = PromptTemplate(
    input_variables=["question", "thoughts"],
    template="""Question: {question}

Let's approach this step-by-step:

{thoughts}

Based on these thoughts, what is the final answer?
"""
)

# Define examples for few-shot learning
examples = [
    {
        "question": "What is 15 + 27?",
        "thoughts": """1) First, let's break down the numbers: 15 is 10 + 5, and 27 is 20 + 7.
2) We can rearrange these: (10 + 20) + (5 + 7)
3) 10 + 20 = 30
4) 5 + 7 = 12
5) So now we have 30 + 12
6) 30 + 12 = 42"""
    },
    {
        "question": "How many months have 28 days?",
        "thoughts": """1) Let's think about this carefully. At first, we might think only February has 28 days.
2) However, the question is asking how many months HAVE 28 days, not how many ONLY have 28 days.
3) Every month has at least 28 days.
4) Therefore, all 12 months have at least 28 days."""
    }
]

# Create the few-shot prompt template
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=PromptTemplate(
        input_variables=["question", "thoughts"],
        template="Question: {question}\n\nThoughts: {thoughts}"
    ),
    prefix="Here are some examples of breaking down questions into steps:",
    suffix="Now, let's approach a new question:\n\n{input}",
    input_variables=["input"],
    example_separator="\n\n"
)

# Combine the few-shot examples with the base prompt
chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate(
        input_variables=["input"],
        template=few_shot_prompt.format(input=base_prompt.format(question="{question}", thoughts="{thoughts}"))
    )
)

# Use the chain
question = "If a train travels 120 km in 2 hours, what is its average speed in km/h?"
result = chain.run(question=question, thoughts="Let's break this down step by step:")

print(result)

To find the average speed of the train, we need to divide the total distance traveled by the time it took.

1) The train traveled 120 km.
2) It took 2 hours to travel this distance.
3) So, we divide 120 by 2.
4) 120 / 2 = 60.

The final answer is that the train's average speed is 60 km/h.
